In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pickle
import plotly as py
from plotly.offline import plot, iplot
from scipy import stats
import plotly.graph_objs as go
from sklearn.cluster import DBSCAN
py.offline.init_notebook_mode(connected=True)

%matplotlib notebook

In [2]:
# path in the file
path_to_data = "/home/samba693/DataChallenge/debs2019_dataset2/"
path_to_outliers = "/home/samba693/Documents/DEBS/debs2019/src/ssh-kd/data/outliers.pkl"

In [3]:
# Read in the file
df = pd.read_csv(path_to_data+"in.csv",usecols=[1,2,3,4],names=["laser_id","X","Y","Z"])
num_of_scenes = len(df) / 72000
dataframes = []

for i in range(int(num_of_scenes)):
    start = i * 72000
    end = start + 72000
    dataframes.append(df.iloc[start:end, :])

In [4]:

out_file = open(path_to_data+"out.csv", 'r').readlines()
outfile = [i.rstrip() for i in out_file]
outfile = [i.split(',') for i in outfile]
outfile = [i[1:] for i in outfile]

outfile_list = []
out_cluster = []
for i in outfile:
    a  = {}
    for j in range(0,len(i),2):
        a[i[j]] = int(i[j+1])
    out_cluster.append(sum(a.values()))
    outfile_list.append(a)

In [5]:
# remove the outliers
def remove_out(df,path_out = path_to_outliers):
    pd.options.mode.chained_assignment = None
    outliers = pd.read_pickle(path_out)
    max_rad = outliers[0]
    min_rad = outliers[1]

    df["radius"] = df.X.pow(2).add(df.Y.pow(2).add(df.Z.pow(2))).pow(0.5).round(1)
    rad = np.array(df.radius)
    bool_vec = (rad <= max_rad) & (rad >= min_rad)
#     bool_vec = (rad >= min_rad)
    df = df[~bool_vec]
    df.drop(df[df["radius"] == 0].index, inplace=True)
#     df = df[df['laser_id']>30]
    
    return df

In [6]:
def plot_3d(df):
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='3d')
    ax.scatter(df['X'],df['Z'],df['Y'], marker='o')

In [7]:
def plot_plot(object_data_frame,labels=False):
    if labels:
        labels1 = tuple(object_data_frame['labels'].tolist())
        use_labels = labels1
    else:
        use_labels = 'black'
    x = tuple(object_data_frame['X'].tolist())
    y = tuple(object_data_frame['Y'].tolist())
    z = tuple(object_data_frame['Z'].tolist())
    
    trace = go.Scatter3d(
            x = x,
            y = z,
            z = y,
            mode = 'markers',
            marker = dict(
                        color=use_labels,
                        colorscale='Viridis',
                        size = 1,
                        opacity = 0.8)
            )
    layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        nticks=0, range = [-150,150],),
                    yaxis = dict(
                        nticks=0, range = [-100,100],),
                    zaxis = dict(
                        nticks=0, range = [-100,100],),),
                  )
    data=[trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [8]:
# used for plots for paper

# temp = dataframes[0]
# temp = temp[(temp['X']>=0) & (temp['Y']<=20) & (temp['Z']>=0)]
# plot_plot(temp)

# temp1 = remove_out(dataframes[0])
# temp1 = temp1[(temp1['X']>=0) & (temp1['Y']<=20) & (temp1['Z']>=0)]
# plot_plot(temp1)

# # clustering with DBSCAN and removing noise
# def doClusteringWithDBSCAN(data):    
#     clustering = DBSCAN(eps=1, min_samples=16).fit(data)
#     labels = clustering.labels_
#     return labels
# data =  np.array(
#                 list(
#                     zip(
#                         np.array(temp1['X']), 
#                         np.array(temp1['Y']), 
#                         np.array(temp1['Z'])
#                     )
#                 )
#             )

# labels = doClusteringWithDBSCAN(data)

# temp1['labels'] = labels

# temp2 = temp1[~(temp1['labels']==-1)]

# plot_plot(temp2,True)


In [9]:
def removeOutWithDBSCAN(temp1):
    data =  np.array(
                list(
                    zip(
                        np.array(temp1['X']), 
                        np.array(temp1['Y']), 
                        np.array(temp1['Z'])
                    )
                )
            )
    clustering = DBSCAN(eps=1, min_samples=16).fit(data)
    labels = clustering.labels_
    return temp1[~(labels==-1)]

In [10]:
import math
# calculating degree
calDegrees=lambda x : round(math.degrees(math.atan(x)), 1)
vfunc = np.vectorize(calDegrees)
def tang(x,y):
    """ Get Tan inverse of y/x to get the angle."""
    tag= np.divide(y, x)
    return tag


def get_degrees(c):
    """ Get the degree for the data based upon cordinate plane.
    Tan behaviour in cordinate system.
    1st cord: theta
    2nd cord: 180+theta
    3rd cord: 180+theta
    4th cord: 360+theta
    return: degree of dataframe rows
    """
    
    if c['X']==0 and c['Z']>0:
        return 90
    if c['X']==0 and c['Z']<0:
        return 270
    
    degrees =  vfunc(tang(c['X'],c['Z']))
    if c['X']<0:
        degrees = 180+degrees
    if (c['X']>0 and c['Z']<0):
        degrees = 360+degrees
        
    return degrees

def angle_of_elevation(x,y,z):
    """Get Tan inverse of y/sqrt(x^2+z^2) to get the angle"""
    den = math.sqrt(x**2+z**2)
    etan= np.divide(y,den)
    return etan 


def get_phi(c):
    """ Get the degree for the data based upon cordinate plane.
    return: degree of dataframe rows
    """
    phi =  vfunc(angle_of_elevation(c['X'],c['Y'],c['Z']))
    phi =  90-phi
    return np.float(phi)

def get_r(x,y,z):
    """Get density r"""
    den = math.sqrt(x**2+z**2)
    return den 

def get_den(c):
    r = get_r(c['X'],c['Y'],c['Z'])
    return r

In [11]:
# Using Dataframe apply, a func is run on dataframe, pass the datafram to get_degrees

def get_different_sectors(temp, threshold=0.7):
    temp['angles'] = temp.apply(get_degrees,axis=1)
    temp['top_rad'] = temp.apply(get_den,axis=1)
    temp.angles = temp.angles.astype(np.float)
    unique_angles = sorted(list(set(temp.angles.value_counts().index)))
    indexes_to_split = list(np.where(np.diff(unique_angles)>=threshold)[0]+1)
    start=0
    if len(unique_angles)==0:
        angle_ranges = [unique_angles]
    else:
        angle_ranges = []
    for i in indexes_to_split:
        angle_ranges.append(unique_angles[start:i])
        start = i
    list_of_sectors = []
    angles = np.array(temp.angles)
    for j in angle_ranges:
        max_angle = max(j)
        min_angle = min(j)
        bool_vec = (angles>=min_angle) & (angles<=max_angle)
        list_of_sectors.append(temp[bool_vec])
    list_of_valid = []
    for i,j in enumerate(list_of_sectors):
        if len(j)>10:
            j['labels'] = i
            list_of_valid.append(j)
    return list_of_valid

def get_valid_density(list_of_valid_sec):
    list_of_valid_density = []
    for temp in list_of_valid_sec:
        # Initializing the kernel
        kernel = stats.gaussian_kde(temp['top_rad'].to_numpy(),bw_method=0.05)

        # Evaluating the values
        to_plot2 = kernel.evaluate(np.linspace(-20,180,500))
        # plt.plot(np.linspace(-20,180,500),to_plot2)
        # Threshold ==0.01
        bool_vec = [~(to_plot2<=0.001)]

        # Selecting valid values wrt threshold
        to_plot = to_plot2[bool_vec]
        x_val = np.linspace(-20,180,500)[bool_vec]

        # Selecting the boundary points
        req_indexes = np.where((np.diff(x_val)<=0.5)==False)[0]+1
        markers_on = x_val[req_indexes].round(0).astype(int).tolist()
        markers_on = [0]+markers_on+[180]

        # Calculate the dense indexs
        to_dense = np.split(to_plot,req_indexes)
        try:
            max_dense = [np.max(j) for j in to_dense]
        except:
#             print(x_val)
#             print(to_dense)
#             print(req_indexes)
            list_of_valid_density.append(temp)

    #     print(markers_on)
    #     print(max_dense)

        # Selecting the valid objects
        for i in range(len(markers_on)-1):
            if max_dense[i]>=0.01:
                temp1 = temp[ (temp['top_rad']>=markers_on[i]) & (temp['top_rad']<=markers_on[i+1]) ]
                if len(temp1)>=15:
                    list_of_valid_density.append(temp1)
    return list_of_valid_density

In [12]:
# plot the different sectors
def plot_sectors(list_of_sec,labels = True):
    df = pd.concat(list_of_sec)
    plot_plot(df, labels)

In [13]:
# Radius density
# temp = list_of_valid_sec[13]
# temp['top_rad'] = (temp['top_rad']**0.5).round(1)
# temp['top_rad'].plot.kde(bw_method=0.1,figsize=(10,10))

In [14]:
# # plot_sectors([list_of_valid_density[2]],True)
# # plot_sectors(list_of_valid_density,True)
# # plot_sectors([list_of_valid_sec[2]],True)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

count=0
for i in dataframes:
    list_of_valid_sec = get_different_sectors(remove_out(i))
    list_of_valid_density = get_valid_density(list_of_valid_sec)
    print("original clusters = {}, number of detected = {}".format(out_cluster[count],len(list_of_valid_density)))
    count+=1

# list_of_valid_sec = get_different_sectors(remove_out(dataframes[0]))
# list_of_valid_density = get_valid_density(list_of_valid_sec)
# print("original clusters = {}, number of detected = {}".format(out_cluster[0],len(list_of_valid_density)))

original clusters = 33, number of detected = 35
original clusters = 25, number of detected = 30
original clusters = 25, number of detected = 25
original clusters = 17, number of detected = 18
original clusters = 16, number of detected = 22
original clusters = 34, number of detected = 33
original clusters = 35, number of detected = 34
original clusters = 20, number of detected = 25
original clusters = 17, number of detected = 22
original clusters = 32, number of detected = 30
original clusters = 39, number of detected = 26
original clusters = 47, number of detected = 32
original clusters = 40, number of detected = 22
original clusters = 12, number of detected = 16
original clusters = 19, number of detected = 19
original clusters = 26, number of detected = 25
original clusters = 16, number of detected = 11
original clusters = 30, number of detected = 35
original clusters = 28, number of detected = 30
original clusters = 45, number of detected = 32
original clusters = 22, number of detect

original clusters = 38, number of detected = 31
original clusters = 42, number of detected = 37
original clusters = 21, number of detected = 22
original clusters = 28, number of detected = 23
original clusters = 40, number of detected = 35
original clusters = 10, number of detected = 11
original clusters = 16, number of detected = 20
original clusters = 21, number of detected = 26
original clusters = 27, number of detected = 30
original clusters = 15, number of detected = 23
original clusters = 17, number of detected = 21
original clusters = 34, number of detected = 29
original clusters = 25, number of detected = 27
original clusters = 24, number of detected = 24
original clusters = 42, number of detected = 32
original clusters = 46, number of detected = 39
original clusters = 14, number of detected = 20
original clusters = 23, number of detected = 21
original clusters = 33, number of detected = 30
original clusters = 45, number of detected = 36
original clusters = 39, number of detect

original clusters = 27, number of detected = 30
original clusters = 33, number of detected = 28
original clusters = 12, number of detected = 17
original clusters = 45, number of detected = 43
original clusters = 36, number of detected = 30
original clusters = 27, number of detected = 19
original clusters = 38, number of detected = 37
original clusters = 12, number of detected = 18
original clusters = 36, number of detected = 39
original clusters = 29, number of detected = 23
original clusters = 11, number of detected = 12
original clusters = 10, number of detected = 19
original clusters = 28, number of detected = 33
original clusters = 41, number of detected = 42
original clusters = 10, number of detected = 14
original clusters = 34, number of detected = 29
original clusters = 39, number of detected = 34
original clusters = 37, number of detected = 37
original clusters = 41, number of detected = 38
original clusters = 10, number of detected = 11
original clusters = 43, number of detect

In [15]:
# for i in range(len(markers_on)-1):    
#     basic_plot3 = basic_plot2[ (basic_plot2['X']>=markers_on[i]) & (basic_plot2['X']<=markers_on[i+1])]
#     plt.plot(basic_plot3['X'].to_numpy(),basic_plot3['Y'].to_numpy())

# out_cluster[3]
plot_sectors(list_of_valid_density,True)

In [16]:
# for i in range(len(markers_on)-1):    
#     temp1 = temp[ (temp['top_rad']>=markers_on[i]) & (temp['top_rad']<=markers_on[i+1]) ]
#     if len(temp1)>=20:
#         list_of_valid_density.append(temp1)
plot_plot(remove_out(dataframes[499]))

In [23]:
plot_sectors([list_of_valid_density[11]],True)

In [18]:
plot_sectors([list_of_valid_sec[13]], True)
print(out_cluster[3])
print(outfile_list[3])
print(len(list_of_valid_sec))

17
{'ToyotaPriusSimple': 4, 'ClothRecyclingContainer': 1, 'DrinkingFountain': 1, 'EmergencyPhone': 1, 'FireHydrant': 2, 'OldBench': 1, 'PhoneBooth': 1, 'PublicBin': 1, 'UndergroundContainer': 2, 'BigSassafras': 1, 'Sassafras': 1, 'ScooterSimple': 1}
26


In [19]:
# # return the unique phi and angle
# def get_phi_sectors(list_of_valid_sec,thresold = 0.6):
#     list_of_sec_phi = []
#     for ij in list_of_valid_sec:
#         ij['phi'] = ij.apply(get_phi,axis=1)
#         unique_phi = sorted(list(set(ij.phi.value_counts().index)))
#         indexes_to_split = list(np.where(np.diff(unique_phi)>=thresold)[0]+1)
#         start=0
#         if len(indexes_to_split)==0:
#             phi_ranges = [unique_phi]
#         else:
#             phi_ranges = []
#         for i in indexes_to_split:
#             phi_ranges.append(unique_phi[start:i])
#             start = i

#         angles = np.array(ij.phi)
#         for j in phi_ranges:
#             max_angle = max(j)
#             min_angle = min(j)
#             bool_vec = (angles>=min_angle) & (angles<=max_angle)
#             list_of_sec_phi.append(ij[bool_vec])

#     list_of_valid_sec_phi = []
#     for i,j in enumerate(list_of_sec_phi):
#         if len(j)>10:
#             j['labels'] = i
#             list_of_valid_sec_phi.append(j)
#     return list_of_valid_sec_phi

In [20]:
# list_of_valid_sec_phi = get_phi_sectors(list_of_valid_sec)

In [21]:
# plot_sectors([list_of_valid_sec_phi[13]],True)
# print(len(list_of_valid_sec_phi))

In [22]:
# # return the unique phi and angle
# def get_valid_r(list_of_sec_phi, thresold=2):
#     list_of_r = []
#     for ij in list_of_valid_sec_phi:
#         ij['r'] = ij.apply(get_den,axis=1)
#         temp = ij.r.round(1).value_counts()
#         unique_r = sorted(list(set(temp[temp>5].index)))
#         indexes_to_split = list(np.where(np.diff(unique_r)>=thresold)[0]+1)
#         start=0
#         if len(indexes_to_split)==0:
#             phi_ranges = [unique_r]
#         else:
#             phi_ranges = []
#         for i in indexes_to_split:
#             phi_ranges.append(unique_r[start:i])
#             start = i

#         angles = np.array(ij.r)
#         for j in phi_ranges:
#             if len(j)!=0:
# #                 print(j)
#                 max_angle = max(j)
#                 min_angle = min(j)
#                 bool_vec = (angles>=min_angle) & (angles<=max_angle)
#                 list_of_r.append(ij[bool_vec])

#     list_of_valid_r = []
#     for i,j in enumerate(list_of_r):
#         if len(j)>10:
#             j['labels'] = i
#             list_of_valid_r.append(j)
#     return list_of_valid_r

In [23]:
# list_of_valid_r = get_valid_r(list_of_valid_sec_phi)

In [24]:
# plot_sectors(list_of_valid_r)
# # plot_sectors([list_of_valid_r[0]])
# print(len(list_of_valid_r[0]))

In [25]:
# # segmenting intot clusters
# def segment(df):
#     objects = []
#     for i,j in enumerate(df):
#         temp = get_valid_r(get_phi_sectors(get_different_sectors(remove_out(j))))
#         objects.append(temp)
#         print("original clusters = {}, number of detected = {}".format(out_cluster[i],len(temp)))
#     return objects

In [24]:
# list_of_objects = segment(dataframes)

In [34]:
list_of_valid_sec[10]

,laser_id,X,Y,Z,radius,angles,top_rad,labels
35995877,37,-13.602068,-1.916161,-43.724964,45.8,252.7,45.791798,28
35995878,38,-4.614693,-0.768460,-14.834309,15.6,252.7,15.535511,28
35995879,39,-4.605173,-0.885094,-14.803707,15.5,252.7,15.503463,28
35995880,40,-4.598331,-1.001926,-14.781713,15.5,252.7,15.480429,28
35995881,41,-4.591563,-1.118540,-14.759957,15.5,252.7,15.457645,28
35995882,42,-4.584406,-1.234831,-14.736950,15.5,252.7,15.433550,28
35995883,43,-4.577435,-1.350952,-14.714540,15.5,252.7,15.410081,28
35995884,44,-4.566284,-1.465530,-14.678696,15.4,252.7,15.372543,28
35995885,45,-4.561361,-1.581861,-14.662868,15.4,252.7,15.355967,28
35995941,37,-13.779144,-1.906900,-43.437366,45.6,252.4,45.570490,28


In [43]:
z = list_of_valid_sec[9]['Z'].to_numpy()
x = list_of_valid_sec[9]['X'].to_numpy()

In [48]:
to_check = np.round(np.arctan2(z,x)*180/np.pi,1)
to_check[to_check<0] = to_check[to_check<0]+360

In [49]:
original = list_of_valid_sec[9]['angles'].to_numpy()
to_check==original

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [50]:
original

array([220.7, 220.7, 220.7, 220.4, 220.4, 220.4, 220.4, 220.4, 220.1,
       220.1, 220.1, 220.1, 220.1, 219.8, 219.8, 219.8, 219.8, 219.8,
       219.8, 219.8, 219.4, 219.4, 219.4, 219.4, 219.4, 219.4, 219.4,
       219.1, 219.1, 219.1, 219.1, 218.8, 218.8, 221.7, 221.7, 221.7,
       221.4, 221.4, 221.4, 221.4, 221. , 221. , 221. , 221. , 221. ,
       221. ])

In [51]:
to_check

array([220.7, 220.7, 220.7, 220.4, 220.4, 220.4, 220.4, 220.4, 220.1,
       220.1, 220.1, 220.1, 220.1, 219.8, 219.8, 219.8, 219.8, 219.8,
       219.8, 219.8, 219.4, 219.4, 219.4, 219.4, 219.4, 219.4, 219.4,
       219.1, 219.1, 219.1, 219.1, 218.8, 218.8, 221.7, 221.7, 221.7,
       221.4, 221.4, 221.4, 221.4, 221. , 221. , 221. , 221. , 221. ,
       221. ])

In [52]:
np.unique(to_check)

array([218.8, 219.1, 219.4, 219.8, 220.1, 220.4, 220.7, 221. , 221.4,
       221.7])

In [55]:
np.sum(to_check==220.7)

3

In [58]:
temppppp = np.hstack([original.reshape(-1,1),to_check.reshape(-1,1),original.reshape(-1,1)])

In [59]:
np.hstack([temppppp,original.reshape(-1,1),to_check.reshape(-1,1)])

array([[220.7, 220.7, 220.7, 220.7, 220.7],
       [220.7, 220.7, 220.7, 220.7, 220.7],
       [220.7, 220.7, 220.7, 220.7, 220.7],
       [220.4, 220.4, 220.4, 220.4, 220.4],
       [220.4, 220.4, 220.4, 220.4, 220.4],
       [220.4, 220.4, 220.4, 220.4, 220.4],
       [220.4, 220.4, 220.4, 220.4, 220.4],
       [220.4, 220.4, 220.4, 220.4, 220.4],
       [220.1, 220.1, 220.1, 220.1, 220.1],
       [220.1, 220.1, 220.1, 220.1, 220.1],
       [220.1, 220.1, 220.1, 220.1, 220.1],
       [220.1, 220.1, 220.1, 220.1, 220.1],
       [220.1, 220.1, 220.1, 220.1, 220.1],
       [219.8, 219.8, 219.8, 219.8, 219.8],
       [219.8, 219.8, 219.8, 219.8, 219.8],
       [219.8, 219.8, 219.8, 219.8, 219.8],
       [219.8, 219.8, 219.8, 219.8, 219.8],
       [219.8, 219.8, 219.8, 219.8, 219.8],
       [219.8, 219.8, 219.8, 219.8, 219.8],
       [219.8, 219.8, 219.8, 219.8, 219.8],
       [219.4, 219.4, 219.4, 219.4, 219.4],
       [219.4, 219.4, 219.4, 219.4, 219.4],
       [219.4, 219.4, 219.4, 219